<a href="https://colab.research.google.com/github/izzul112/ML_BMLT-Proyek_Akhir-Sistem_Rekomendasi-Anime/blob/main/Proyek_Akhir_Sistem_Rekomendasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Laporan Proyek Machine Learning - Habib Azizul Haq**

# Project Overview

Sistem Rekomendasi Anime

Content-based Filtering

# Business Understanding

**Problem Statements**
Menjelaskan pernyataan masalah latar belakang:

*   Dari serangkaian fitur yang ada, fitur apa yang paling berpengaruh terhadap kelas harga suatu hp?
*   Berapa kelas harga pasar hp dengan karakteristik atau fitur tertentu?

**Goals**
Menjelaskan tujuan dari pernyataan masalah:

*   Mengetahui fitur yang paling berkorelasi dengan kelas harga hp.
*   Membuat model machine learning yang dapat memprediksi kelas harga hp seakurat mungkin berdasarkan fitur-fitur yang ada.

**Solution statements**

*   Untuk menghasilkan model yang optimal namun tetap sederhana kita akan menggunakan 3 algoritma yaitu KNeighborsClassifier, RandomForestClassifier, GradientBoostingClassifier.
*   Dari 3 algoritma di atas kita akan menggunakan metrik Mean Absolute Error yang memang cocok untuk kasus klasifikasi, untuk melihat pada ketiga algoritma diatas mana yang paling powerfull.


# **Data Understanding**

Kita akan menggunakan dataset yang terdiri dari 2 buah file .csv. Yaitu anime.csv yang berisi judul anime dan rating yang berisi rating dari suatu anime.

Sumber dataset yang akan kita gunakan berasal dari [Kaggle](https://www.kaggle.com/datasets/CooperUnion/anime-recommendations-database). Yang di publis oleh COOPERUNION.

Kita lanjut dengan melihat isi variabel dari setiap file kita mulai dari file anime.csv terlebih dahulu yang mempunyai 7 variabel:

*   anime_id : id unik dari situs myanimelist.net yang mengidentifikasi anime.
*   name: nama lengkap anime.
*   genre: daftar genre yang dipisahkan koma untuk anime setiap anime.
*   type: setiap anime memiliki tipe seperti movie, TV, OVA, etc.
*   episodes: berapa jumlah episode dari suatu judul anime, biasanya untuk anime musiman 12-13 episode per musim. (1 jika itu movie).
*   rating: rating rata-rata dari 10 untuk anime.
*   members: jumlah anggota komunitas yang ada di anime ini "group".

Sedangkan rating.csv hanya memiliki 3 variabel:

*   user_id: ID pengguna yang dibuat secara acak yang tidak dapat diidentifikasi.
*   anime_id: anime yang telah dinilai oleh pengguna ini.
*   rating: rating dari 10 yang telah ditetapkan pengguna ini (-1 jika pengguna menontonnya tetapi tidak memberikan rating).

Oh iya semua sumber asli dari dataset diambil dari situs [myanimelist](https://myanimelist.net/), yang merupakan website yang sering di kunjungi bagi pecinta amine dan manga.

Kita akan melakukan sedikit analisis untuk mengethui beberapa hal tentang variabel diatas diantaranya :
1.  Kita akan melihat ada berapa jumlah baris dari data dalam tampilan tabel.
2.  Kita akan melihat tipe data dari 2 file .csv di dataset.
3.  Kita akan menggabungkan kedua dataset.
4.  Kita akan mengganti nilai rating yang -1.
5.  Kita akan melihat apakah ada data yang kosong / tidak ada isinya.
6.  Kita akan melihat apakah data kita memiliki outliers.
7.  Kita akan melihat variabel apa saja yang memiliki hubungan yang kuat atas klasifikasi harga suatu hp.

Sebelum kita mulai mengerjakan kita import semua lubrary yang di butuhkan

In [1]:
# Import library
import pandas as pd
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

Kita sambungkan dulu Google Colab kita dengan Google Drive, karena dataset kita berada di Google Drive.

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Setelah itu kita buat variabel untuk menampung dataset kita:


*   data_anime : untuk menampung data anime.csv
*   data_rating : untuk menampung date rating.csv



In [3]:
data_anime = pd.read_csv('/content/gdrive/My Drive/dataset/anime.csv')

data_rating = pd.read_csv('/content/gdrive/My Drive/dataset/rating.csv')

# **Pembahasan**
**1. Kita akan melihat ada berapa jumlah baris dari data dalam tampilan tabel.**

Kita akan menggunakan menggunakan kode `data_anime`, `data_anime` adalah nama variabel yang kita ganakan saat meload dataset anime.csv.

In [4]:
data_anime

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


Dari kode `data_anime` kita mendapat informasi:

*   Ada 12.294 baris dataset anime.csv kita
*   Dan terdapat 7 kolom

Selanjutnya kita lihat untuk rating.csv dengan kode `data_rating`,`data_rating` adalah nama variabel yang kita ganakan saat meload dataset rating.csv.



In [5]:
data_rating

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
7813732,73515,16512,7
7813733,73515,17187,9
7813734,73515,22145,10
7813735,73516,790,9


Dari kode `data_rating` kita mendapat informasi:

*   Ada 7.813.737 baris dataset rating.csv kita
*   Dan terdapat 3 kolom

**2. Kita akan melihat tipe data dari kedua dataset kita.**

Kita akan menggunakan `data_anime.info()`, `.info()` akan menampilkan informasi tipe data 7 kolom dataset anime.csv yang kita gunakan.

In [6]:
data_anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


Dari *output* `data_anime.info()`, terlihat bahwa:

*   Terdapat 2 kolom dengan tipe data int64, yaitu: anime_id dan members.
*   Terdapat 3 kolom dengan tipe data object, yaitu: name, genre, type dan episodes.
*   Dan terdapat 1 kolom dengan tipe data float64, yaitu: rating.

Kita akan menggunakan `data_rating.info()`, untuk menampilkan tipe data dari data rating.csv


In [7]:
data_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB


Dari *output* `data_rating.info()`, terlihat bahwa:

*   Hanya terdapat 1 tipe data saja yaitu int64



**3. Kita akan menggabungkan kedua dataset**

Kita akan menggabungkannya dengan fungsi `.merge()` dari pandas. 

In [8]:
anime_fulldata = pd.merge(data_anime, data_rating, on='anime_id', suffixes= ['_web', '_user'])
anime_fulldata = anime_fulldata.rename(columns={'name': 'judul_anime'})
anime_fulldata.head()

,anime_id,judul_anime,genre,type,episodes,rating_web,members,user_id,rating_user
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,99,5
1,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,152,10
2,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,244,10
3,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,271,10
4,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,278,-1


Jika sudah hasilnya akan seperti tampilan diatas, dengan rating dari file anime.csv berubah menjadi rating_web, kemudian kita juga merubah rating dari file rating.csv menjadi rating_user untuk memudahkan kita dalam membedakan mana rating dari user dan mana yang bukan, kita juga sudah mengganti kolom nama menjadi judul_anime supaya lebih mudah dimengerti.

**4.  Kita akan mengganti nilai rating yang -1.**

Penggantian ini penting mengingat nilai -1 terjadi karena user menonton anime tersebut tapi tidak memberikan rating, maka dari itu supaya nilai -1 ini tidak mempengaruhi sitem rekomendasi kita, kita akan menggantinya dengan nilai NaN / kosong supaya bisa kita hapus baris data yang mengandung nilai -1.

Sebelum itu kita lihat dulu nilai ada berapa jumlah nilai -1 nya.

In [9]:
anime_fulldata.loc[(anime_fulldata['rating_user']==-1)]

,anime_id,judul_anime,genre,type,episodes,rating_web,members,user_id,rating_user
4,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,278,-1
13,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,684,-1
18,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,917,-1
29,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,1133,-1
31,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,1252,-1
...,...,...,...,...,...,...,...,...,...
7813721,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175,27364,-1
7813722,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175,39532,-1
7813723,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175,48766,-1
7813725,26081,Yasuji no Pornorama: Yacchimae!!,Hentai,Movie,1,5.46,142,27364,-1


Dari hasil diatas kita bisa lihat bahwa:
- Ada 1.476.488 baris yang mengandung nilai -1 di kolom rating_user.

Setelah itu kita mengganti nilai -1 dengan fungsi `.replace({-1: np.nan}, inplace=True)`, nilai di dalam `{-1: np.nan}` berarti -1 diganti dengan nilai NaN dan `inplace=True` supaya jika kita berhasil mengganti nilai rating_user mengembalinkan nilai None / tidak ada.

In [10]:
anime_feature = anime_fulldata.copy()
anime_feature["rating_user"].replace({-1: np.nan}, inplace=True)
anime_feature

,anime_id,judul_anime,genre,type,episodes,rating_web,members,user_id,rating_user
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,99,5.0
1,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,152,10.0
2,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,244,10.0
3,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,271,10.0
4,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,278,NaN
...,...,...,...,...,...,...,...,...,...
7813722,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175,39532,NaN
7813723,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175,48766,NaN
7813724,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175,60365,4.0
7813725,26081,Yasuji no Pornorama: Yacchimae!!,Hentai,Movie,1,5.46,142,27364,NaN


Hasil di atas menunjukkan bahwa kita berhasil merubah nilai yang tadinya -1 menjadi NaN.

In [14]:
anime_feature = anime_feature.dropna(axis = 0, how ='any') 
anime_feature.loc[(anime_feature['rating_user']==-1)]

,anime_id,judul_anime,genre,type,episodes,rating_web,members,user_id,rating_user


Dan hasilnya kita berhasil menghapus nilai -1 dari kolom rating_user.

**4.  Kita akan melihat apakah ada data yang kosong / tidak ada isinya.**

In [15]:
anime_feature.isnull().sum()

anime_id       0
judul_anime    0
genre          0
type           0
episodes       0
rating_web     0
members        0
user_id        0
rating_user    0
dtype: int64

In [13]:
anime_feature.count()

anime_id       6337146
judul_anime    6337146
genre          6337146
type           6337146
episodes       6337146
rating_web     6337146
members        6337146
user_id        6337146
rating_user    6337146
dtype: int64